# ⚙️ 02. 전처리 및 피처 엔지니어링

이 노트북에서는 결측치 처리, 범주형 변수 인코딩, 클러스터링, 외부 데이터 병합 등을 수행합니다.

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', 100)

In [ ]:
# 데이터 로드
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')
bus = pd.read_csv('../data/raw/bus_feature.csv')
subway = pd.read_csv('../data/raw/subway_feature.csv')

In [ ]:
# 외부 데이터 병합 (버스/지하철)
train = train.merge(bus, on='ID', how='left')
train = train.merge(subway, on='ID', how='left')
test = test.merge(bus, on='ID', how='left')
test = test.merge(subway, on='ID', how='left')

In [ ]:
# 결측치 처리 (예시)
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [ ]:
# 위치 기반 클러스터링 (KMeans)
coords = train[['경도', '위도']]
kmeans = KMeans(n_clusters=20, random_state=42)
train['location_cluster'] = kmeans.fit_predict(coords)
test['location_cluster'] = kmeans.predict(test[['경도', '위도']])

In [ ]:
# 범주형 변수 인코딩
categorical = ['시군구', '아파트명', '거래유형']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(pd.concat([train[categorical], test[categorical]]))

X_train_cat = pd.DataFrame(encoder.transform(train[categorical]), columns=encoder.get_feature_names_out(categorical))
X_test_cat = pd.DataFrame(encoder.transform(test[categorical]), columns=encoder.get_feature_names_out(categorical))

# 인덱스 정렬
X_train_cat.index = train.index
X_test_cat.index = test.index

# 원본에서 범주형 변수 제거 후 합치기
train = pd.concat([train.drop(columns=categorical), X_train_cat], axis=1)
test = pd.concat([test.drop(columns=categorical), X_test_cat], axis=1)

In [ ]:
# 최종 전처리 결과 저장
train.to_csv('../data/processed/train_processed.csv', index=False)
test.to_csv('../data/processed/test_processed.csv', index=False)